In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

# Place dataset path here
FNNpath = "../Datasets/FNN Titles/"
CoAIDpath = "../Datasets/CoAID/"


politiFactFake = pd.read_csv(FNNpath+"politifact_fake.csv", usecols=['title'])
politiFactFake['label']=0
gossipCopFake = pd.read_csv(FNNpath+"gossipcop_fake.csv", usecols=['title'])
gossipCopFake['label']=0
politiFactTrue = pd.read_csv(FNNpath+"politifact_real.csv", usecols=['title'], nrows=len(politiFactFake.values))
politiFactTrue['label']=1
gossipCopTrue = pd.read_csv(FNNpath+"gossipcop_real.csv", usecols=['title'],  nrows=len(gossipCopFake.values))
gossipCopTrue['label']=1

CoAIDFalse = pd.read_csv(CoAIDpath+"NewsFakeCOVID-19.csv", usecols=['title'])
CoAIDFalse['label']=0
CoAIDTrue = pd.read_csv(CoAIDpath+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(CoAIDFalse.values))
CoAIDTrue['label']=1


dfTotal = pd.concat([politiFactTrue, gossipCopTrue, politiFactFake, gossipCopFake, CoAIDTrue, CoAIDFalse])
X = dfTotal['title'].values
y = dfTotal['label'].values

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
  X, y, test_size=0.25, random_state=450)



path_external = "/content/drive/MyDrive/CoVerifi&MedVerifi/" # Replace with appropriate path
validationNewsDF = pd.read_csv(path_external+"Combined_News.csv", usecols=['title', 'label'])


def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0

validationNewsDF['label'] = validationNewsDF['label'].apply(lambda label: binaryLabel(str(label)))

validationX = validationNewsDF['title'].values
validationX = [one_hot(str(elem),10000) for elem in validationX]
validationX = pad_sequences(validationX, padding='post', maxlen=500)
validationY = validationNewsDF['label'].values

In [4]:
#For SVM, LR, and NB, we used the default settings provided in the scikit-learn 
#For CNN we use the standard implementation with default setting https://github.com/dennybritz/cnn-text-classification-tf
#Could just use an out-of-the-box CNN implementation from a TF tutorial
# The dennybritz one seems good but I can do it later
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
# Bernoulli was best of above
import math
X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

svmClassifier = svm.SVC()
svmClassifier.fit(X_train, y_train_base)
svmPrediction = svmClassifier.predict(X_test)
svmPrediction = [math.floor(0.5+pred) for pred in svmPrediction]
print("CoAID_and_all_of_FNN: Results of SVM internal validation:")
print(classification_report(y_test_base, svmPrediction))

svmValidationPrediction = svmClassifier.predict(validationX)
svmValidationPrediction = [math.floor(0.5+pred) for pred in svmValidationPrediction]
print("CoAID_and_all_of_FNN: Results of SVM external validation, on our new dataset:")
print(classification_report(validationY, svmValidationPrediction))



X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

lrClassifier = LogisticRegression(random_state=0, max_iter=500)
lrClassifier.fit(X_train, y_train_base)
lrPrediction = lrClassifier.predict(X_test)
lrPrediction = [math.floor(0.5+pred) for pred in lrPrediction]
print("CoAID_and_all_of_FNN: Results of LR internal validation:")
print(classification_report(y_test_base, lrPrediction))

lrValidationPrediction = lrClassifier.predict(validationX)
lrValidationPrediction = [math.floor(0.5+pred) for pred in lrValidationPrediction]
print("CoAID_and_all_of_FNN: Results of LR external validation, on our new dataset:")
print(classification_report(validationY, lrValidationPrediction))

X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

bnbClassifier = BernoulliNB()
bnbClassifier.fit(X_train, y_train_base)
bnbPrediction = bnbClassifier.predict(X_test)
bnbPrediction = [math.floor(0.5+pred) for pred in bnbPrediction]
print("CoAID_and_all_of_FNN: Results of Bernoulli NB internal validation:")
print(classification_report(y_test_base, bnbPrediction))
bnbValidationPrediction = bnbClassifier.predict(validationX)
bnbValidationPrediction = [math.floor(0.5+pred) for pred in bnbValidationPrediction]
print("CoAID_and_all_of_FNN: Results of Bernoulli NB external validation, on our new dataset:")
print(classification_report(validationY, bnbValidationPrediction))



CoAID_and_all_of_FNN: Results of SVM internal validation:
              precision    recall  f1-score   support

           0       0.53      0.47      0.50      1557
           1       0.54      0.60      0.56      1607

    accuracy                           0.53      3164
   macro avg       0.53      0.53      0.53      3164
weighted avg       0.53      0.53      0.53      3164

CoAID_and_all_of_FNN: Results of SVM external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.65      0.57      0.61      3883
           1       0.55      0.63      0.58      3175

    accuracy                           0.60      7058
   macro avg       0.60      0.60      0.60      7058
weighted avg       0.61      0.60      0.60      7058



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


CoAID_and_all_of_FNN: Results of LR internal validation:
              precision    recall  f1-score   support

           0       0.51      0.56      0.54      1557
           1       0.53      0.48      0.51      1607

    accuracy                           0.52      3164
   macro avg       0.52      0.52      0.52      3164
weighted avg       0.52      0.52      0.52      3164

CoAID_and_all_of_FNN: Results of LR external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      3883
           1       0.52      0.50      0.51      3175

    accuracy                           0.57      7058
   macro avg       0.56      0.56      0.56      7058
weighted avg       0.57      0.57      0.57      7058

CoAID_and_all_of_FNN: Results of Bernoulli NB internal validation:
              precision    recall  f1-score   support

           0       0.53      0.29      0.37      1557
           1       0.52      0.75   